JOINS | LIT | COALESCE | UNION | INTERSECT | withColumnRenamed

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql.functions import coalesce, lit

In [5]:
conf = pyspark.SparkConf().setAppName("joinsspark").setMaster("local")
sc = SparkSession.builder.config(conf=conf).getOrCreate()

25/01/14 16:11:11 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 192.168.20.45 instead (on interface wlp0s20f3)
25/01/14 16:11:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/14 16:11:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data1 = [(1, "John"),
         (2, "Jane"),
         (3, "Alice")]

data2 = [(1, "Math"),
         (2, "Science"),
         (4, "History")]

In [7]:
df1 = sc.createDataFrame(data1, ["id", "name"])
df2 = sc.createDataFrame(data2, ["id", "name"])

In [8]:
df1.show()
df2.show()

+---+-----+
| id| name|
+---+-----+
|  1| John|
|  2| Jane|
|  3|Alice|
+---+-----+

+---+-------+
| id|   name|
+---+-------+
|  1|   Math|
|  2|Science|
|  4|History|
+---+-------+



In [9]:
left_df = df1.join(df2, on="id", how="left")
left_df.show()

+---+-----+-------+
| id| name|   name|
+---+-----+-------+
|  1| John|   Math|
|  3|Alice|   null|
|  2| Jane|Science|
+---+-----+-------+



In [10]:
right_df = df1.join(df2, df1.id == df2.id, "right")
right_df.select(df1["id"], df2["name"]).show()


+----+-------+
|  id|   name|
+----+-------+
|   1|   Math|
|   2|Science|
|null|History|
+----+-------+



In [11]:
right_df.withColumnRenamed("id", "identifier").show()
# .withColumnRenamed(df2["name"], "subject")
# new_df.select("identifier", coalesce("subject", "none")).show()

+----------+----+----------+-------+
|identifier|name|identifier|   name|
+----------+----+----------+-------+
|         1|John|         1|   Math|
|         2|Jane|         2|Science|
|      null|null|         4|History|
+----------+----+----------+-------+



In [12]:
new_df = right_df.select(df1["id"], coalesce(df1["name"], lit("none")).alias("subject")).withColumnRenamed("id", "identifier")
# .withColumnRenamed("name", "subject") this doesn't work because the column name is not the same
new_df.show()

+----------+-------+
|identifier|subject|
+----------+-------+
|         1|   John|
|         2|   Jane|
|      null|   none|
+----------+-------+



In [14]:
data1 = [(1, "John"),
         (2, "Jane"),
         (3, "Alice")]

data2 = [(3, "Alice"),
         (4, "Bob"),
         (5, "Charlie")]

df1 = sc.createDataFrame(data1, ["ID", "Name"])
df2 = sc.createDataFrame(data2, ["ID", "Name"])

In [15]:
df1.union(df2).show()
# Both work same in spark
df1.unionAll(df2).show()

+---+-------+
| ID|   Name|
+---+-------+
|  1|   John|
|  2|   Jane|
|  3|  Alice|
|  3|  Alice|
|  4|    Bob|
|  5|Charlie|
+---+-------+

+---+-------+
| ID|   Name|
+---+-------+
|  1|   John|
|  2|   Jane|
|  3|  Alice|
|  3|  Alice|
|  4|    Bob|
|  5|Charlie|
+---+-------+



In [16]:
df1.intersect(df2).show()

+---+-----+
| ID| Name|
+---+-----+
|  3|Alice|
+---+-----+



In [17]:
df1.subtract(df2).show()

+---+----+
| ID|Name|
+---+----+
|  2|Jane|
|  1|John|
+---+----+

